<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3 Interlude: Chisel Standard Library
**Prev: [Generators: Collections](3.2_collections.ipynb)**<br>
**Next: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**

## Motivation
Chisel is all about re-use, so it only makes sense to provide a standard library of interfaces (encouraging interoperability of RTL) and generators for commonly-used hardware blocks.

## Setup

In [1]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

Compiling d:\Academics\PhD\4.Chisel\chisel-bootcamp\Main.sc

Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/maven-metadata.xml
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3_2.12/3.4.4/chisel3_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3-macros_2.12/3.4.4/chisel3-macros_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/chisel3-core_2.12/3.4.4/chisel3-core_2.12-3.4.4.pom
Downloaded https://repo1.maven.org/maven2/edu/berkeley/cs/firrtl_2.12/1.4.4/firrtl_2.12-1.4.4.pom
Downloaded https://repo1.maven.org/maven2/net/jcazevedo/moultingyaml_2.12/0.4.2/moultingyaml_2.12-0.4.2.pom
Downloaded https://repo1.maven.org/maven2/org/json4s/json4s-native_2.12/3.6.9/json4s-native_2.12-3.6.9.pom
Downloaded https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.7.1/antlr4-runtime-4.7.1.pom
Downloaded https://repo1.maven.org/maven2/com/google/protobuf/protobuf-java/3.9.0/protobuf-java-3.9.0.pom
Downloaded https://repo1.maven.org/maven2/org/apache/commons/commons-

Compiling d:\Academics\PhD\4.Chisel\chisel-bootcamp\Main.sc #2

path: String = "d:\\Academics\\PhD\\4.Chisel\\chisel-bootcamp/source/load-ivy.sc"

In [2]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

import chisel3._

import chisel3.util._

import chisel3.tester._

import chisel3.tester.RawTester.test

---
# The Cheatsheet
The [Chisel3 cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf) contains a summary of all the major hardware construction APIs, including some of the standard library utilities that we'll introduce below.

# Decoupled: A Standard Ready-Valid Interface

Data transfering element : Source ,
Data receiving element : Sink

One of the commonly used interfaces provided by Chisel is `DecoupledIO`, providing a ready-valid interface for transferring data. The idea is that the source drives the `bits` signal with the data to be transferred and the `valid` signal when there is data to be transferred. The sink drives the `ready` signal when it is ready to accept data, and data is considered transferred when both `ready` and `valid` are asserted on a cycle.

This provides a flow control mechanism in both directions for data transfer, including a backpressure mechanism.

Note: `ready` and `valid` should not be combinationally coupled, otherwise this may result in unsynthesizable combinational loops. `ready` should only be dependent on whether the sink is able to receive data, and `valid` should only be dependent on whether the source has data. Only after the transaction (on the next clock cycle) should the values update.

Any Chisel data can be wrapped in a `DecoupledIO` (used as the `bits` field) as follows:

```scala
val myChiselData = UInt(8.W)
// or any Chisel data type, such as Bool(), SInt(...), or even custom Bundles
val myDecoupled = Decoupled(myChiselData)
```



The above creates a new `DecoupledIO` Bundle with fields
- `valid`: Output(Bool) - set to true by the source when the valid data is on the bits wire
- `ready`: Input(Bool) - set to true by the sink when it is ready to receive the data
- `bits`: Output(UInt(8.W)) - used to transfer data

- when both ready and valid are true in the same clock cycle, the data is considered to be transferred.

- we shouldnt couple ready ad valid signals using combinational operators otherwise it creates a loop
___

The rest of the section will be structured somewhat differently from the ones before: instead of giving you coding exercises, we're going to give some code examples and testcases that print the circuit state. Try to predict what will be printed before just running the tests.

## Queues

`Queue` creates a FIFO (first-in, first-out) queue with Decoupled interfaces on both sides, allowing backpressure. Both the data type and number of elements are configurable. 

backpressure is a flow control mechanism that slows down or stops the data producer when the consumer can’t keep up.

>Notice how the first few times we print out the `peek`ed value we call `litValue` on it. This converts the chisel literal that is returned into a `BigInt`. Later similar calls we do not call `litValue` and you can see extra information about the value `peek` returns. like the type and the width.

![](images/queue.png)

In [3]:
test(new Module {
    // Example circuit using a Queue
    val io = IO(new Bundle {

      // creating an input to the queue
      // note that the flipped will filp the bits of the decopled io
      // Flipped(...): Reverses the direction of the DecoupledIO.
      // Normally, a Decoupled interface is an output (producer).
      // By flipping it, you turn it into an input (consumer-facing)
      val in = Flipped(Decoupled(UInt(8.W)))

      // creating an output of the queue
      val out = Decoupled(UInt(8.W))
    })
    
    // A 2-element FIFO queue is created from io.in, and its output is connected to io.out
    // This Queue uses Decoupled handshake (i.e., valid and ready signals)
    // io.in: producer trying to push data into the queue.
    // io.out: consumer trying to read from the queue.
    val queue = Queue(io.in, 2)  // 2-element queue

    // io.out used as the output of the queue
    io.out <> queue
  }) { c =>

    c.io.out.ready.poke(false.B)  // source(queue) will stop 
    c.io.in.valid.poke(true.B)  // Enqueue an element
    c.io.in.bits.poke(42.U)
    println(s"Starting:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}") // in.ready will be 1 as there are space in the queue
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")  // out.valid will be 0 as there are no elements
    c.clock.step(1)

    c.io.in.valid.poke(true.B)  // Enqueue another element
    c.io.in.bits.poke(43.U)
    // What do you think io.out.valid and io.out.bits will be?
    println(s"After first enqueue:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}") // in.ready will be 1 as there are one more space in the queue
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}") // out.valid will be 1 as there are valid data(42) in the queue
    c.clock.step(1)

    c.io.in.valid.poke(true.B)  // Read a element, attempt to enqueue
    c.io.in.bits.poke(44.U) // spill since no space in the queue
    c.io.out.ready.poke(true.B)
    // What do you think io.in.ready will be, and will this enqueue succeed, and what will be read?
    println(s"On first read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}") // in.ready will be 0 as there are no space in the queue
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}") // out.valid will be 1 as there are valid data(43,42)
    c.clock.step(1)

    c.io.in.valid.poke(false.B)  // Read elements out
    c.io.out.ready.poke(true.B)
    // What do you think will be read here?
    println(s"On second read:") 
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}") // now there will be one more space since io.out ready is true and 42 is consumed, therefore this will be 1
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}") // io.valid = 1, io.bits = 43
    c.clock.step(1)

    // Will a third read produce anything?
    // there are no elements in the queue
    println(s"On third read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}") // ready = 1 , space available
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}") // valid =0 , bits = garbage value
    c.clock.step(1)
}

Elaborating design...
Done elaborating.
Starting:
	io.in: ready=1
	io.out: valid=0, bits=0
After first enqueue:
	io.in: ready=1
	io.out: valid=1, bits=42
On first read:
	io.in: ready=0
	io.out: valid=1, bits=42
On second read:
	io.in: ready=1
	io.out: valid=1, bits=43
On third read:
	io.in: ready=1
	io.out: valid=0, bits=42
test Helper_Anon Success: 0 tests passed in 7 cycles in 0.043175 seconds 162.13 Hz


## Arbiters
Arbiters route data from _n_ `DecoupledIO` sources to one `DecoupledIO` sink, given a prioritization.

![](images/arbiter_2.png)

There are two types included in Chisel:
- `Arbiter`: prioritizes lower-index producers
- `RRArbiter`: runs in round-robin order

Note that Arbiter routing is implemented in combinational logic.

The below example will demonstrate the use of the priority arbiter (which you will also implement in the next section):

In [14]:
test(new Module {
    // Example circuit using a priority arbiter
    val io = IO(new Bundle {

      // input of arbiter should be flipped since it is a consumer when it comes to input
      // since there are two data sources, we need a vector of inputs of size 2
      val in = Flipped(Vec(2, Decoupled(UInt(8.W))))

      // output of arbiter should not be flipped since it is the producer when it comes to output
      val out = Decoupled(UInt(8.W))
    })

    // Arbiter doesn't have a convenience constructor, so it's built like any Module
    // This arbiter chooses the values from two producers
    val arbiter = Module(new Arbiter(UInt(8.W), 2))  // 2 to 1 Priority Arbiter

    // set the input and the output of the arbiter
    arbiter.io.in <> io.in
    io.out <> arbiter.io.out

  }) { c =>

    // initially valid is false
    c.io.in(0).valid.poke(false.B)
    c.io.in(1).valid.poke(false.B)
    c.io.out.ready.poke(false.B)
    println(s"Start:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")  // since the out.ready = 0 , i.e. outside sink is not ready to receive data, arbiter is not ready as well, therefore ready bits would be 0
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")            // since there are no values inside the arbiter, valid bits would be 0, and bits would be some garbage value(or 0)

    // arbiter only needs combinational logic therefore no clocks
    
    c.io.in(1).valid.poke(true.B)  // Valid input 1
    c.io.in(1).bits.poke(42.U)
    c.io.out.ready.poke(true.B)
    // What do you think the output will be?
    println(s"valid input 1:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")  // out.ready = 1, i.e. sink is ready to accept data so arbiter would set its ready signals to 1
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")  // since source 1(second) has valid data , valid = 1 and out = 42
    
    
    c.io.in(0).valid.poke(true.B)  // Valid inputs 0 and 1
    c.io.in(0).bits.poke(43.U)
    // What do you think the output will be? Which inputs will be ready?
    println(s"valid inputs 0 and 1:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")  // out.ready = 1, i.e. sink is ready to accept data so arbiter would set its ready signals to 1 but now source0 has valid values it will not accept data from source1 hence that ready bit = 0
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")  // now both in0 and in1 has valid input data , but prioratize in0 , therefore valid = 1 , bits = 43
    
    c.io.in(1).valid.poke(false.B)  // Valid input 0
    // What do you think the output will be?
    println(s"valid input 0:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")   // out.ready = 1, i.e. sink is ready to accept data so arbiter would set its ready signals to 1 but now source0 has valid values it will not accept data from source1 hence that ready bit = 0
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}") // eventhough in1 is invalid, in0 is still valid therefore still valid = 1, bits = 43
}

Elaborating design...
Done elaborating.
Start:
	in(0).ready=0, in(1).ready=0
	out.valid=0, out.bits=0
valid input 1:
	in(0).ready=1, in(1).ready=1
	out.valid=1, out.bits=42
valid inputs 0 and 1:
	in(0).ready=1, in(1).ready=0
	out.valid=1, out.bits=43
valid input 0:
	in(0).ready=1, in(1).ready=0
	out.valid=1, out.bits=43
test Helper_Anon Success: 0 tests passed in 2 cycles in 0.005938 seconds 336.84 Hz


# Misc Function Blocks
Chisel Utils has some helpers that perform stateless functions.

## Bitwise Utilities
### PopCount
PopCount returns the number of high (1) bits in the input as a `UInt`.

### Reverse
Reverse returns the bit-reversed input.

In [ ]:
test(new Module {
    // Example circuit using PopCount
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := PopCount(io.in)
  }) { c =>
    // Integer.parseInt is used create an Integer from a binary specification
    c.io.in.poke(Integer.parseInt("00000000", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("00001111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("11001010", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("11111111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

}

In [ ]:
test(new Module {
    // Example circuit using Reverse
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := Reverse(io.in)
  }) { c =>
    // Integer.parseInt is used create an Integer from a binary specification
    c.io.in.poke(Integer.parseInt("01010101", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("00001111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("11110000", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("11001010", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")
}

## OneHot encoding utilities
OneHot is an encoding of integers where there is one wire for each value, and exactly one wire is high. This allows the efficient creation of some functions, for example muxes. However, behavior may be undefined if the one-wire-high condition is not held.

The below two functions provide conversion between binary (`UInt`) and OneHot encodings, and are inverses of each other:
- UInt to OneHot: `UIntToOH`
- OneHot to UInt: `OHToUInt`

In [ ]:
test(new Module {
    // Example circuit using UIntToOH
    val io = IO(new Bundle {
      val in = Input(UInt(4.W))
      val out = Output(UInt(16.W))
    })
    io.out := UIntToOH(io.in)
  }) { c =>
    c.io.in.poke(0.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(1.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(8.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(15.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")
}

In [ ]:
test(new Module {
    // Example circuit using OHToUInt
    val io = IO(new Bundle {
      val in = Input(UInt(16.W))
      val out = Output(UInt(4.W))
    })
    io.out := OHToUInt(io.in)
}) { c =>
    c.io.in.poke(Integer.parseInt("0000 0000 0000 0001".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("0000 0000 1000 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("1000 0000 0000 0001".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    // Some invalid inputs:
    // None high
    c.io.in.poke(Integer.parseInt("0000 0000 0000 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    // Multiple high
    c.io.in.poke(Integer.parseInt("0001 0100 0010 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")
}


## Muxes
These muxes take in a list of values with select signals, and output the value associated with the lowest-index select signal.

These can either take a list of (select: Bool, value: Data) tuples, or corresponding lists of selects and values as arguments. For simplicity, the examples below only demonstrate the second form.

### Priority Mux
A `PriorityMux` outputs the value associated with the lowest-index asserted select signal.

### OneHot Mux
An `Mux1H` provides an efficient implementation when it is guaranteed that exactly one of the select signals will be high. Behavior is undefined if the assumption is not true.

In [ ]:
test(new Module {
    // Example circuit using PriorityMux
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := PriorityMux(io.in_sels, io.in_bits)
  }) { c =>
    c.io.in_bits(0).poke(10.U)
    c.io.in_bits(1).poke(20.U)

    // Select higher index only
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select both - arbitration needed
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select lower index only
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")
}

In [ ]:
test(new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := Mux1H(io.in_sels, io.in_bits)
  }) { c =>
    c.io.in_bits(0).poke(10.U)
    c.io.in_bits(1).poke(20.U)

    // Select index 1
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select index 0
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select none (invalid)
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select both (invalid)
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")
}

## Counter
`Counter` is a counter that can be incremented once every cycle, up to some specified limit, at which point it overflows. Note that it is **not** a Module, and its value is accessible.

In [ ]:
test(new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val count = Input(Bool())
      val out = Output(UInt(2.W))
    })
    val counter = Counter(3)  // 3-count Counter (outputs range [0...2])
    when(io.count) {
      counter.inc()
    }
    io.out := counter.value
  }) { c =>
    c.io.count.poke(true.B)
    println(s"start: counter value=${c.io.out.peek().litValue}")

    c.clock.step(1)
    println(s"step 1: counter value=${c.io.out.peek().litValue}")

    c.clock.step(1)
    println(s"step 2: counter value=${c.io.out.peek().litValue}")

    c.io.count.poke(false.B)
    c.clock.step(1)
    println(s"step without increment: counter value=${c.io.out.peek().litValue}")

    c.io.count.poke(true.B)
    c.clock.step(1)
    println(s"step again: counter value=${c.io.out.peek().litValue}")
}

---
# You're done!

[Return to the top.](#top)